# Data augmentation

Data augmentation is a process by which we modify our images, and masks when we load them from file using the dataset Object. This process is stochasitc so that the images loaded in each training epoch are slightly different.

The main reason one uses data augmentation is to augment the generalization of our model to new images. By changing the images in each training epoch, we force the model to learn more generalizationalbe features instead of learning specific features unique to each labeled image. 


To implement data augmentation, I used the [Albumentations](https://albumentations.ai/) library. One can create a pipeline to process our images when they are loaded from file. The library can work with image-mask pairs used for segmenation. 

The normalization of our images (set mean and std of each channel to 0 and 1) can also be performed by the data augmentation pipeline. See the notebook on data normalization.

Below I am using 4 transformations. We can set the probability that this transformation is applied using the `p` argument. You can set it in the project configuration file. Alternatively, you can edit the code below.

Tips

* If you are tracking left/right body parts, you probably don't want to flip your images.


In [7]:
import albumentations as A
import cv2
from unetTracker.trackingProject import TrackingProject
from unetTracker.dataset import UNetDataset
import pickle
import os

In [2]:
project = TrackingProject(name="mouseTrack",root_folder = "/home/kevin/Documents/trackingProjects/")

Project directory: /home/kevin/Documents/trackingProjects/mouseTrack
Loading /home/kevin/Documents/trackingProjects/mouseTrack/config.yalm
{'augmentation_HorizontalFlipProb': 0.0, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_size': [480, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'mouseTrack', 'normalization_values': {'means': [0.39945241808891296, 0.3994884490966797, 0.39926499128341675], 'stds': [0.11478571593761444, 0.11476266384124756, 0.11492700129747391]}, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (240.0, 255.0, 255.0)], 'objects': ['snout', 'earL', 'earR', 'tail'], 'target_radius': 5}


In [3]:
original_height = project.image_size[0]
original_width = project.image_size[1]
means = project.normalization_values["means"]
stds = project.normalization_values["stds"]

trainTransform = A.Compose([   
                    A.RandomSizedCrop(min_max_height=(original_height-50, original_height),w2h_ratio=original_width/original_height,height=original_height, width=original_width, p=project.augmentation_RandomSizedCropProb),
                    A.HorizontalFlip(p=project.augmentation_HorizontalFlipProb),
                    A.Rotate (limit=30,border_mode=cv2.BORDER_CONSTANT,p=project.augmentation_RotateProb),
                    A.RandomBrightnessContrast(p=project.augmentation_RandomBrightnessContrastProb),
                    A.Normalize(mean=means, std=stds)
])

valTransform = A.Compose([   
                    A.Normalize(mean=means, std=stds)
])


print(trainTransform)
print(valTransform)

Compose([
  RandomSizedCrop(always_apply=False, p=1.0, min_max_height=(430, 480), height=480, width=480, w2h_ratio=1.0, interpolation=1),
  HorizontalFlip(always_apply=False, p=0.0),
  Rotate(always_apply=False, p=0.3, limit=(-30, 30), interpolation=1, border_mode=0, value=None, mask_value=None, rotate_method='largest_box', crop_border=False),
  RandomBrightnessContrast(always_apply=False, p=0.2, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
  Normalize(always_apply=False, p=1.0, mean=[0.39945241808891296, 0.3994884490966797, 0.39926499128341675], std=[0.11478571593761444, 0.11476266384124756, 0.11492700129747391], max_pixel_value=255.0),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})
Compose([
  Normalize(always_apply=False, p=1.0, mean=[0.39945241808891296, 0.3994884490966797, 0.39926499128341675], std=[0.11478571593761444, 0.11476266384124756, 0.11492700129747391], max_pixel_value=255.0),
], p=1.0, bbox_params=None, keyp

We can save the transformation in a `augmentation` directory inside our project directory.

In [4]:
project.augmentation_dir

'/home/kevin/Documents/trackingProjects/mouseTrack/augmentation'

In [9]:
fileName = os.path.join(project.augmentation_dir,"trainTransform")
print("Saving trainTransform as", fileName)
pickle.dump( trainTransform, open( fileName, "wb" ))


Saving trainTransform as /home/kevin/Documents/trackingProjects/mouseTrack/augmentation/trainTransform


In [10]:
fileName = os.path.join(project.augmentation_dir,"valTransform")
print("Saving trainTransform as", fileName)
pickle.dump( valTransform, open( fileName, "wb" ))


Saving trainTransform as /home/kevin/Documents/trackingProjects/mouseTrack/augmentation/valTransform
